# CHSH Game

In [6]:
# Imports

from qiskit import QuantumCircuit
from qiskit_aer.primitives import Sampler
from numpy import pi
from numpy.random import randint

def chsh_game(strategy):
    """ Playing the game
    Args:
        strats(callable): A func that makes 2 bits (int's) and returns 
        2 bits (int's as well). Strats must follow the rules of CHSH
        Game.
    Returns:
        int: 1 for a win, 0 for a loss 
    """
    
    # ref chooses x and y randomly
    x, y = randint(0, 2), randint(0, 2)

    # Strategy to use 
    a, b = strategy(x, y)

    # ref decides if A or B wins the game
    if (a != b) == (x & y):
        return 1 # winner
    return 0 # loser

def chsh_circuit(x, y):
    """ Creation of QuantumCitrcuit` that implements the best strategy
    Args:
        x (int): A's bit(must be 0 or 1)
        y (int): B's bit(must be 0 or 1)
    Returns:
        QuantumCircuit: Circuit that, when runs, returns A and B's
           answer bits
    """
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.barrier()

    # A
    if x == 0:
        qc.ry(0, 0)
    else:
        qc.ry(-pi / 2, 0)
    qc.measure(0, 0)

    # B
    if y == 0:
        qc.ry(-pi / 4, 1)
    else:
        qc.ry(pi / 4, 1)
    qc.measure(1, 1)

    return qc

print("(x, y) = (0, 0)")
display(chsh_circuit(0, 0).draw())

print("(x, y) = (0, 1)")
display(chsh_circuit(0, 1).draw())

print("(x, y) = (1, 0)")
display(chsh_circuit(1, 0).draw())

print("(x, y) = (1, 1)")
display(chsh_circuit(1, 1).draw())

(x, y) = (0, 0)


┌───┐      ░  ┌───────┐  ┌─┐   
q_0: ┤ H ├──■───░──┤ Ry(0) ├──┤M├───
     └───┘┌─┴─┐ ░ ┌┴───────┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─░─┤ Ry(-π/4) ├─╫─┤M├
          └───┘ ░ └──────────┘ ║ └╥┘
c: 2/══════════════════════════╩══╩═
                               0  1

(x, y) = (0, 1)


┌───┐      ░  ┌───────┐ ┌─┐   
q_0: ┤ H ├──■───░──┤ Ry(0) ├─┤M├───
     └───┘┌─┴─┐ ░ ┌┴───────┴┐└╥┘┌─┐
q_1: ─────┤ X ├─░─┤ Ry(π/4) ├─╫─┤M├
          └───┘ ░ └─────────┘ ║ └╥┘
c: 2/═════════════════════════╩══╩═
                              0  1

(x, y) = (1, 0)


┌───┐      ░ ┌──────────┐┌─┐   
q_0: ┤ H ├──■───░─┤ Ry(-π/2) ├┤M├───
     └───┘┌─┴─┐ ░ ├──────────┤└╥┘┌─┐
q_1: ─────┤ X ├─░─┤ Ry(-π/4) ├─╫─┤M├
          └───┘ ░ └──────────┘ ║ └╥┘
c: 2/══════════════════════════╩══╩═
                               0  1

(x, y) = (1, 1)


┌───┐      ░ ┌──────────┐┌─┐   
q_0: ┤ H ├──■───░─┤ Ry(-π/2) ├┤M├───
     └───┘┌─┴─┐ ░ ├─────────┬┘└╥┘┌─┐
q_1: ─────┤ X ├─░─┤ Ry(π/4) ├──╫─┤M├
          └───┘ ░ └─────────┘  ║ └╥┘
c: 2/══════════════════════════╩══╩═
                               0  1

In [8]:
sampler = Sampler()

def quantum_strategy(x, y):
    """Carry out the best strategy for the CHSH game.
    Args:
        x (int): Alice's bit (must be 0 or 1)
        y (int): Bob's bit (must be 0 or 1)
    Returns:
        (int, int): Alice and Bob's answer bits (respectively)
    """

    # `shot=1 runs the circuit`
    result = sampler.run(chsh_circuit(x, y), shots=1).result()
    statistics = result.quasi_dists[0].binary_probabilities()
    bits = list(statistics.keys())[0]
    a, b = bits[0], bits[1]
    return a, b

In [9]:
NUM_GAMES = 1000
TOTAL_SCORE = 0

for _ in range(NUM_GAMES):
    TOTAL_SCORE += chsh_game(quantum_strategy)

print("Fraction of games won", TOTAL_SCORE / NUM_GAMES)

Fraction of games won 0.852


In [10]:
def clssical_strategy(x, y):
    """An optimal classical strategy for the CHSH game
    Args:
        x (int): Alice's bit (must be 0 or 1)
        y (int): Bob's bit (must be 0 or 1)
    Returns:
        (int, int): Alice and Bob's answer bits (respectively)
    """
    # A's answer
    if x == 0:
        a = 0
    elif x == 1:
        a = 1
    
    # B's answe
    if x == 0:
        b = 1
    elif x == 1:
        b = 0
    return a, b

In [12]:
NUM_GAMES = 1000
TOTAL_SCORE = 0

for _ in range(NUM_GAMES):
    TOTAL_SCORE += chsh_game(clssical_strategy)

print("Fraction of games won", TOTAL_SCORE / NUM_GAMES)

Fraction of games won 0.269
